In [1]:
from gurobipy import *
import numpy as np
import time
import random

# ----- Données du problème -----
n = 5 # nombre d'agents (lignes)
m = 10 # nombre d'items (colonnes)

# Bornes pour les agents et items
#lower_agent = [1] * n # li
#upper_agent = [m] * n # ui
lower_item  = [1] * m # lj_prime
upper_item  = [1] * m # uj_prime

# Matrice des coûts aléatoires
c = np.random.randint(1, 1000, size=(n, m))
#c = np.array([[5,8,4,9,7],[1,3,2,7,8],[3,9,2,9,5],[10,1,3,3,4],[5,1,7,7,3]]) # Exemple du polycopié
##print("La matrice des couts c :")
##for i in range(n):
##    print("i =", i, "|", c[i])

def fct_w(n):
    return np.array([n - k for k in range(n)])

def fct_w_prime(w):
    n = len(w)
    w_prime = np.zeros(n)
    for k in range(n - 1):
        w_prime[k] = w[k] - w[k + 1]
    w_prime[n - 1] = w[n - 1]
    return w_prime

w = fct_w(n)
w_prime = fct_w_prime(w)

In [3]:
start = time.time()
# ----- Création du modèle -----
model = Model("assignment")
# Pour éviter d'avoir trop d'output
model.Params.OutputFlag = 0

# Variables :
x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x")
b = model.addVars(n, n, vtype=GRB.CONTINUOUS, lb=0, name="b")
r = model.addVars(n, vtype=GRB.CONTINUOUS, name="r")

# Contraintes pour chaque agent (la somme des x de chaque agent doit être comprise entre lower_agent et upper_agent)
#for i in range(n):
#    model.addConstr(quicksum(x[i, j] for j in range(m)) >= lower_agent[i],
#                    name=f"agent_lower_{i}")
#    model.addConstr(quicksum(x[i, j] for j in range(m)) <= upper_agent[i],
#                    name=f"agent_upper_{i}")

# Contraintes pour chaque item (chaque item doit être assigné exactement à 1 agent)
for j in range(m):
    model.addConstr(quicksum(x[i, j] for i in range(n)) >= lower_item[j],
                    name=f"item_lower_{j}")
    model.addConstr(quicksum(x[i, j] for i in range(n)) <= upper_item[j],
                    name=f"item_upper_{j}")

# Contrainte liant r et b aux coûts d'assignation
for i in range(n):
    for k in range(n):
        model.addConstr(r[k] + b[i, k] >= quicksum(c[i, j] * x[i, j] for j in range(m)),
                        name=f"c3_{i}_{k}")

# Construction de l'objectif (il est re-définie à chaque itération)
#obj_expr = (quicksum(w_prime[k] * (k + 1) * r[k] for k in range(n)) + quicksum(b[i, k] for i in range(n) for k in range(n)))
obj_expr = quicksum(w_prime[k]*((k+1)*r[k] + quicksum(b[i,k] for i in range(n))) for k in range(n))
model.setObjective(obj_expr, GRB.MINIMIZE)

# ----- Arrondi Itératif -----
# À chaque itération, on résout le LP, puis on fixe le x[i,j] non encore fixée (c'est-à-dire dont lb < ub)
# qui possède la plus grande valeur (le plus grand x_{ij} dans la solution courante) en le contraignant à 1.
iteration = 0
max_iterations = n * m # borne de sécurité
#time_tot = 0
#somme_1 = [0 for i in range(n)]
somme_2 = [0 for j in range(m)]

while iteration < max_iterations:
    #model.update()

    #model.write("arrondi_iter1.lp")
    model.optimize()
    #time_tot += model.RUNTIME
    #print("TEST", model.Runtime)

    # Si le modèle n'est pas optimal (ou est infaisable), on arrête
    if model.status != GRB.OPTIMAL:
        print("Modèle pas optimal ou infaisable") # normalement cela sort avant, sinon, il y a un problème
        break

    # Prints
    for i in range(n):
        for j in range(m):
            if x[i, j].LB != x[i, j].UB:
                val = x[i, j].X
                print("val = ", val, "i,j :", i, j)
    print("--------------")

    # Recherche de la variable non fixée ayant la plus grande valeur dans la solution courante
    max_val = -1
    sel_i, sel_j = -1, -1
    for i in range(n):
        for j in range(m):
            # On considère uniquement les variables encore non fixées
            if x[i, j].LB != x[i, j].UB: # donc on ne pourrait pas avoir 0 comme plus grande valeur
                val = x[i, j].X
                if val > max_val:
                    #print("VAL :", val)
                    max_val = val
                    sel_i, sel_j = i, j
    print("VAL FIN A :", sel_i, sel_j, max_val)

    if sel_i == -1 or sel_j == -1:
        print("Aucune variable trouvée, fin de l'algorithme")
        break

    #sortir = 0

    # Conditions :

    #somme_1 = 1 + sum(x[sel_i, j].X for j in range(m) if x[sel_i, j].LB == x[sel_i, j].UB)
    #somme_2 = 1 + sum(x[i, sel_j].X for i in range(n) if x[i, sel_j].LB == x[i, sel_j].UB)
    ##print("sel_i =", sel_i, "et somme_1 =", somme_1)
    ##print("sel_j =", sel_j, "et somme_2 =", somme_2)
    #if somme_1 > upper_agent[sel_i]:
    #    print("PROBLEME 1 pour i =", sel_i)
    #    sortir = 1
    #if somme_2 > upper_item[sel_j]:
    #    print("PROBLEME 2 pour j =", sel_j)
    #    sortir = 1

    # A priori c'est impossible que somme_1 dépasse la borne sup, donc, je vérifie pas. Mais, avec d'autres valeurs, il faut vérifier
    #max_somme1 = 0
    #for i in range(n):
    #    if (i == sel_i):
    #        somme_1 = 1 + sum(x[i, j].X for j in range(m) if x[i, j].LB == x[i, j].UB)
    #    else: 
    #        somme_1 = sum(x[i, j].X for j in range(m) if x[i, j].LB == x[i, j].UB)
    #    if somme_1 > max_somme1:
    #        max_somme1 = somme_1
    #    if somme_1 > upper_agent[i]:
    #        print("PROBLEME 1 pour i =", i)
    #        sortir = 1
    #print("max_somme1 =", max_somme1)
                
    #max_somme2 = 0
    #for j in range(m):
    #    if (j == sel_j):
    #        somme_2 = 1 + sum(x[i, j].X for i in range(n) if x[i, j].LB == x[i, j].UB)
    #    else:
    #        somme_2 = sum(x[i, j].X for i in range(n) if x[i, j].LB == x[i, j].UB)
    #    if somme_2 > max_somme2:
    #        max_somme2 = somme_2
    #    if somme_2 > upper_item[j]:
    #        print("PROBLEME 2 pour j =", j)
    #        sortir = 1
    #        
    #print("max_somme2 =", max_somme2)

    #if sortir:
    #    break

    # Fixer la variable sélectionnée à 1 en restreignant sa borne inférieure et sa borne supérieure
    x[sel_i, sel_j].lb = 1
    x[sel_i, sel_j].ub = 1
    model.update() #ca update mais en échange on perd les valeurs .X
    ##print(f"Itération {iteration}: x[{sel_i},{sel_j}] est fixée à 1 (valeur = {max_val})")

    # on pourrait faire 2 listes somme_1 et somme_2 qui stockerait les valeurs des variables fixées
    # on aurait juste à incrémenter de 1
    #somme_1 = sum(1 for j in range(m) if x[sel_i, j].LB != 0) # +1 en dehors de sum si pas de model.update()
    #somme_2 = sum(1 for i in range(n) if x[i, sel_j].LB != 0)
    
    #somme_1[sel_i] += 1
    somme_2[sel_j] += 1
    #print("somme_2 =", somme_2)
    #if (somme_1[sel_i] >= upper_agent[sel_i]):
    #    for j in range(m):
    #        if (x[sel_i, j].LB == 0):
    #            x[sel_i, j].ub = 0 # on fixe x à 0
    if (somme_2[sel_j] >= upper_item[sel_j]):
        for i in range(n):
            if (x[i, sel_j].LB == 0):
                x[i, sel_j].ub = 0 # on fixe x à 0
    
    iteration += 1

end = time.time()

# ----- Affichage de la solution -----
mod_approx_max = model.objVal
##print()
#print("RUNTIME (en s) :", time_tot)
time_max = end-start
print("time_max :", time_max)
print('Valeur de la fonction objectif: %f' % mod_approx_max)
#solution = {(i, j): x[i, j].X for i in range(n) for j in range(m)}
#print("Solution d'assignation:")
#for i in range(n):
#    row = [x[i, j].X for j in range(m)]
#    print(f"Agent {i}: {row}")

##z = [sum(int(c[i,j])*x[i, j].X for j in range(m)) for i in range(n)]
##print("Vecteur z :", z)
#total_cost = sum(c[i][j] * x[i, j].X for i in range(n) for j in range(m))
##total_cost = sum(z)
##print("Coût total =", total_cost)


Toutes les variables sont fixées.
time_rdm : 0.13785624504089355


In [2]:
start = time.time()
# ----- Création du modèle -----
model = Model("assignment")
# Pour éviter d'avoir trop d'output
model.Params.OutputFlag = 0

# Variables :
x = model.addVars(n, m, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x")
b = model.addVars(n, n, vtype=GRB.CONTINUOUS, lb=0, name="b")
r = model.addVars(n, vtype=GRB.CONTINUOUS, name="r")

# Contraintes pour chaque agent (la somme des x de chaque agent doit être comprise entre lower_agent et upper_agent)
#for i in range(n):
#    model.addConstr(quicksum(x[i, j] for j in range(m)) >= lower_agent[i],
#                    name=f"agent_lower_{i}")
#    model.addConstr(quicksum(x[i, j] for j in range(m)) <= upper_agent[i],
#                    name=f"agent_upper_{i}")

# Contraintes pour chaque item (chaque item doit être assigné exactement à 1 agent)
for j in range(m):
    model.addConstr(quicksum(x[i, j] for i in range(n)) >= lower_item[j],
                    name=f"item_lower_{j}")
    model.addConstr(quicksum(x[i, j] for i in range(n)) <= upper_item[j],
                    name=f"item_upper_{j}")

# Contrainte liant r et b aux coûts d'assignation
for i in range(n):
    for k in range(n):
        model.addConstr(r[k] + b[i, k] >= quicksum(c[i, j] * x[i, j] for j in range(m)),
                        name=f"c3_{i}_{k}")

# Construction de l'objectif (il est re-définie à chaque itération)
#obj_expr = (quicksum(w_prime[k] * (k + 1) * r[k] for k in range(n)) + quicksum(b[i, k] for i in range(n) for k in range(n)))
obj_expr = quicksum(w_prime[k]*((k+1)*r[k] + quicksum(b[i,k] for i in range(n))) for k in range(n))
model.setObjective(obj_expr, GRB.MINIMIZE)

# ----- Arrondi Itératif -----
# À chaque itération, on résout le LP, puis on fixe le x[i,j] non encore fixée (c'est-à-dire dont lb < ub)
# qui possède la plus grande valeur (le plus grand x_{ij} dans la solution courante) en le contraignant à 1.
iteration = 0
max_iterations = n * m # borne de sécurité
#time_tot = 0
#somme_1 = [0 for i in range(n)]
somme_2 = [0 for j in range(m)]

while iteration < max_iterations:
    #model.update()

    #model.write("arrondi_iter1.lp")
    model.optimize()
    #time_tot += model.RUNTIME
    #print("TEST", model.Runtime)

    # Si le modèle n'est pas optimal (ou est infaisable), on arrête
    if model.status != GRB.OPTIMAL:
        print("Modèle pas optimal ou infaisable") # normalement cela sort avant, sinon, il y a un problème
        break

    # Prints
    for i in range(n):
        for j in range(m):
            if x[i, j].LB != x[i, j].UB:
                val = x[i, j].X
                print("val = ", val, "i,j :", i, j)
    print("--------------")

    # Recherche de la variable non fixée ayant la plus grande valeur dans la solution courante
    max_val = -1
    sel_i, sel_j = -1, -1
    for i in range(n):
        for j in range(m):
            # On considère uniquement les variables encore non fixées
            if x[i, j].LB != x[i, j].UB: # donc on ne pourrait pas avoir 0 comme plus grande valeur
                val = x[i, j].X
                if val == max_val:
                    #print("OK EGALITE MOMENT")
                    if (c[i, j] < c[sel_i, sel_j]):
                        #print("DIFFERENCE AVEC MAX 1 DETECTEE")
                        #max_val = val
                        #print("VAL 1 :", val)
                        sel_i, sel_j = i, j
                else:
                    if val > max_val:
                        #print("VAL 2 :", val)
                        max_val = val
                        sel_i, sel_j = i, j

    print("VAL FIN B :", sel_i, sel_j, max_val)

    if sel_i == -1 or sel_j == -1:
        print("Aucune variable trouvée, fin de l'algorithme")
        break

    #sortir = 0

    # Conditions :

    #somme_1 = 1 + sum(x[sel_i, j].X for j in range(m) if x[sel_i, j].LB == x[sel_i, j].UB)
    #somme_2 = 1 + sum(x[i, sel_j].X for i in range(n) if x[i, sel_j].LB == x[i, sel_j].UB)
    ##print("sel_i =", sel_i, "et somme_1 =", somme_1)
    ##print("sel_j =", sel_j, "et somme_2 =", somme_2)
    #if somme_1 > upper_agent[sel_i]:
    #    print("PROBLEME 1 pour i =", sel_i)
    #    sortir = 1
    #if somme_2 > upper_item[sel_j]:
    #    print("PROBLEME 2 pour j =", sel_j)
    #    sortir = 1

    # A priori c'est impossible que somme_1 dépasse la borne sup, donc, je vérifie pas. Mais, avec d'autres valeurs, il faut vérifier
    #max_somme1 = 0
    #for i in range(n):
    #    if (i == sel_i):
    #        somme_1 = 1 + sum(x[i, j].X for j in range(m) if x[i, j].LB == x[i, j].UB)
    #    else: 
    #        somme_1 = sum(x[i, j].X for j in range(m) if x[i, j].LB == x[i, j].UB)
    #    if somme_1 > max_somme1:
    #        max_somme1 = somme_1
    #    if somme_1 > upper_agent[i]:
    #        print("PROBLEME 1 pour i =", i)
    #        sortir = 1
    #print("max_somme1 =", max_somme1)
                
    #max_somme2 = 0
    #for j in range(m):
    #    if (j == sel_j):
    #        somme_2 = 1 + sum(x[i, j].X for i in range(n) if x[i, j].LB == x[i, j].UB)
    #    else:
    #        somme_2 = sum(x[i, j].X for i in range(n) if x[i, j].LB == x[i, j].UB)
    #    if somme_2 > max_somme2:
    #        max_somme2 = somme_2
    #    if somme_2 > upper_item[j]:
    #        print("PROBLEME 2 pour j =", j)
    #        sortir = 1
    #        
    #print("max_somme2 =", max_somme2)

    #if sortir:
    #    break

    # Fixer la variable sélectionnée à 1 en restreignant sa borne inférieure et sa borne supérieure
    x[sel_i, sel_j].lb = 1
    x[sel_i, sel_j].ub = 1
    model.update() #ca update mais en échange on perd les valeurs .X
    ##print(f"Itération {iteration}: x[{sel_i},{sel_j}] est fixée à 1 (valeur = {max_val})")

    # on pourrait faire 2 listes somme_1 et somme_2 qui stockerait les valeurs des variables fixées
    # on aurait juste à incrémenter de 1
    #somme_1 = sum(1 for j in range(m) if x[sel_i, j].LB != 0) # +1 en dehors de sum si pas de model.update()
    #somme_2 = sum(1 for i in range(n) if x[i, sel_j].LB != 0)
    
    #somme_1[sel_i] += 1
    somme_2[sel_j] += 1
    #print("somme_2 =", somme_2)
    #if (somme_1[sel_i] >= upper_agent[sel_i]):
    #    for j in range(m):
    #        if (x[sel_i, j].LB == 0):
    #            x[sel_i, j].ub = 0 # on fixe x à 0
    if (somme_2[sel_j] >= upper_item[sel_j]):
        for i in range(n):
            if (x[i, sel_j].LB == 0):
                x[i, sel_j].ub = 0 # on fixe x à 0
    
    iteration += 1

end = time.time()

# ----- Affichage de la solution -----
mod_approx_max_min_cout = model.objVal
##print()
#print("RUNTIME (en s) :", time_tot)
time_max_min_cout = end-start
print("time_max_min_cout :", time_max_min_cout)
print('Valeur de la fonction objectif: %f' % mod_approx_max_min_cout)
#solution = {(i, j): x[i, j].X for i in range(n) for j in range(m)}
#print("Solution d'assignation:")
#for i in range(n):
#    row = [x[i, j].X for j in range(m)]
#    print(f"Agent {i}: {row}")

##z = [sum(int(c[i,j])*x[i, j].X for j in range(m)) for i in range(n)]
##print("Vecteur z :", z)
#total_cost = sum(c[i][j] * x[i, j].X for i in range(n) for j in range(m))
##total_cost = sum(z)
##print("Coût total =", total_cost)


Set parameter Username
Set parameter LicenseID to value 2652794
Academic license - for non-commercial use only - expires 2026-04-15
Aucune variable trouvée, fin de l'algorithme
time_max : 0.11867189407348633


In [4]:
#def sol_equitable_opti(c,w_prime,n,m,l_prime,u_prime,l,u):
def sol_equitable_opti(c,w_prime,n,m,l_prime,u_prime):
    opt_mod2 = Model(name = "min W")
    opt_mod2.Params.OutputFlag = 0
    
    b = opt_mod2.addVars(n, n, vtype = GRB.CONTINUOUS, name = "b", lb = 0)
    r = opt_mod2.addVars(n, name = 'r', vtype = GRB.CONTINUOUS)
    x = opt_mod2.addVars(n, m, vtype = GRB.BINARY, name = "x")
    
    opt_mod2.addConstrs((l_prime[j] <= sum(x[i,j] for i in range(n)) for j in range(m)), name = 'c1a')
    opt_mod2.addConstrs((sum(x[i,j] for i in range(n)) <= u_prime[j] for j in range(m)), name = 'c1b')
    #opt_mod2.addConstrs((l[i] <= sum(x[i,j] for j in range(m)) for i in range(n)), name = 'c2a')
    #opt_mod2.addConstrs((sum(x[i,j] for j in range(m)) <= u[i] for i in range(n)), name = 'c2b')
    
    opt_mod2.addConstrs((r[k] + b[i,k] >= sum(c[i,j]*x[i,j] for j in range(m)) for i in range(n) for k in range(n)), name = 'c3')

    #w_prime = fct_w_prime(w)
    
    obj_fn2 = quicksum(w_prime[k]*((k+1)*r[k] + quicksum(b[i,k] for i in range(n))) for k in range(n))
    opt_mod2.setObjective(obj_fn2, GRB.MINIMIZE)

    return opt_mod2, x

start = time.time()
opt_mod2, x1 = sol_equitable_opti(c,w_prime,n,m,lower_item,upper_item)
opt_mod2.optimize()
end = time.time()
time_exact = end-start
##print()
##print("RUNTIME (en s) :", opt_mod2.RUNTIME)
print("time_exact :", time_exact)
##opt_mod2.write("sol_exacte_poly.lp")

mod_exact = opt_mod2.objVal

#print("Temps total pris pour résoudre le LP de l'algo exact en secondes :", time_opt_mod2) 
#print(f"Nodes time: {nodes_time}")
print('Valeur de la fonction objectif: %f' % mod_exact)

#for i in range(n):
#    row = [z1[i, j].X for j in range(m)]
#    print(f"Agent {i}: {row}")

##z1 = [sum(int(c[i,j])*x1[i, j].X for j in range(m)) for i in range(n)]
#print("Vecteur z1 :", z1)
#total_cost = sum(c[i][j] * z[i, j].X for i in range(n) for j in range(m))
##total_cost1 = sum(z1)
##print("Coût total =", total_cost1)

time_exact : 0.2850799560546875


In [5]:
#print("Temps avec algo max :", time_max)
#print("Temps avec algo random :", time_rdm)
#print("Temps avec algo exact :", time_exact)
ratio1 = mod_approx_max/mod_exact
print("Avec max, Ratio de", ratio1) # Recherche d'un facteur d'approximation
ratio2 = mod_approx_max_min_cout/mod_exact
print("Avec max et min cout, Ratio de", ratio2) # Recherche d'un facteur d'approximation
ratio12 = mod_approx_max/mod_approx_max_min_cout
print("max1/max2, Ratio de", ratio12)
print("---------")

Avec max, Ratio de 1.153586940242624
Avec rdm, Ratio de 5.633218511307473
rdm/max, Ratio de 4.883219733852645
---------
